In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:

!pip install selenium

!apt-get update

!apt install chromium-chromedriver

!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from urllib.parse import quote_plus
import sys
import math
from tqdm import trange, notebook
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert

In [58]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

# 로그인 후 진입


In [59]:
ID = '776177'
PW = 'tnwls2612!'

# 로그인 
login_url='https://www.jobkorea.co.kr'
driver.get(login_url)
driver.find_element(By.XPATH,'//*[@id="point"]/div/div[1]/div[1]/ul/li[1]/button/span[1]').click()
driver.find_element(By.NAME,'M_ID').send_keys(ID)
driver.find_element(By.NAME,'M_PWD').send_keys(PW)
driver.find_element(By.XPATH,'//*[@id="loginForm"]/fieldset/div[1]/button/span').click()

# 로그인 후 it 자기소개서 첫 페이지 진입
driver.get("https://www.jobkorea.co.kr/starter/PassAssay?schPart=10016&schWork=&schEduLevel=&schCType=&schGroup=&isSaved=1&isFilterChecked=1&OrderBy=0&schTxt=")

In [60]:
# 총 자기소개서 개수
# '총 1,025 건'
selfs_num=driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[4]/h4/span[2]').text
selfs_num

# '1,025'
result = selfs_num[2:-1]
num_list = result.split(",")

# '1025'
str_num=""
for num in num_list:
  str_num += num

# 1025
result_num = int(str_num)
result_num

# 1025 / 20
page_num = math.ceil(result_num / 20)
page_num

52

# for 문 try


In [61]:
# 페이지 탐색을 위한 base url
base_url='https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&schPart=10016&isSaved=1&Page='

# 회사 이름 리스트
company_list=list()

# 직무 리스트
field_list=list()

# 합격자 정보 리스트
spec_lists=list()

# 질문 리스트
questions=list()

# 본문 리스트
contents=list()

# 자기소개서 페이지 순회
for page in range(page_num): # page_num
  url = base_url + str(page+1) # page+1

  driver.get(url)
  time.sleep(1)

  # 한 페이지에 존재하는 자기소개서 리스트 개수
  selfLists=driver.find_element(By.CLASS_NAME, 'selfLists')
  self_list_elements=selfLists.find_elements(By.TAG_NAME, 'li') # 20개 or 그 이하
  list_num = len(self_list_elements)
  
  # 한 페이지 리스트 순회
  for li in notebook.tqdm(range(list_num)): #list_num

    # page click
    element=driver.find_element(By.XPATH, (f'//*[@id="container"]/div[2]/div[5]/ul/li[{li+1}]/div[1]/p/a/span'))
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)

    # get 회사 이름 & 직무
    try:
      hd_element=driver.find_element(By.CLASS_NAME, 'hd')

      company_text=hd_element.find_element(By.TAG_NAME, 'a').text
      field_text=hd_element.find_element(By.TAG_NAME, 'em').text

      company_list.append(company_text)
      field_list.append(field_text)

      # get 합격자 정보 - n년제, 학과, 학점
      spec=list()
      for s in range(3):
        spec_text=driver.find_element(By.XPATH, (f'//*[@id="container"]/div[2]/div[1]/div[2]/ul/li[{s+1}]')).text
        spec.append(spec_text)

      spec_lists.append(spec)

      # get 질문 개수
      qnaLists = driver.find_element(By.CLASS_NAME,'qnaLists')
    
      dt = qnaLists.find_elements(By.TAG_NAME,'dt') # question elements
      dd = qnaLists.find_elements(By.TAG_NAME,'dd') # content elements

      q_num = len(dt)
      time.sleep(0.3)

      # get 질문 & 본문
      question = list()
      content = list()

      for q in range(q_num):

        if(dt[q].get_attribute('class') == 'next'):
          dt[q].click()
        # question
        q_element = dt[q].find_element(By.CLASS_NAME, 'tx')
        question.append(q_element.text+"//")

        # content
        c_element = dd[q].find_element(By.CLASS_NAME, 'tx')
        content.append(c_element.text+"//")


      questions.append(question)
      contents.append(content)
      
      # 현재 자기소개서 페이지 나가기 (목록 페이지로 이동)
      driver.get(url)
      time.sleep(1)
    except UnexpectedAlertPresentException:
      driver.get(url)
      time.sleep(1)
      



  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [62]:
#dataframe 생성
a={'company':company_list}
b={'field':field_list}
c={'spec':spec_lists}
d={'question':questions}
e={'content': contents}

A = pd.DataFrame.from_dict(a)
B = pd.DataFrame.from_dict(b)
C = pd.DataFrame.from_dict(c)
D = pd.DataFrame.from_dict(d)
E = pd.DataFrame.from_dict(e)

result = pd.concat([A,B,C,D,E],axis=1)
result.head(n=5)

,company,field,spec,question,content
0,LG CNS,2022년 상반기 신입 웹프로그래머,"[서울4년, 컴퓨터공학과, 학점 4.1/4.5]",[My Competency 지원분야와 관련된 구체적인 지식이나 경험을 기술해주십시오...,[저는 학부 연구생을 하면서 연구실 홈페이지 개발 및 관리를 맡은 경험이 있습니다....
1,현대IT&E,2022년 상반기 신입 웹프로그래머,"[지방4년, 컴퓨터공학부, 학점 3.48/4.5]",[현대백화점그룹이 지향하는 4가지 실천가치(열정/소통/창의/파트너십 중 택1)와 부...,[열정 - 목표를 정하고 이를 달성하기 위해 몰입하는 과정을 통해 성과를 낸 경험이...
2,현대엔지비(주),2022년 상반기 신입 웹프로그래머,"[지방4년, 컴퓨터공학부, 학점 3.48/4.5]","[지원동기 및 포부//, 성장과정//, 성격의 장/단점 및 특기//, 자기PR//]",[현대자동차그룹의 미래 IT 인재를 위한 교육을 책임지는 현대엔지비에서 Softee...
3,구글코리아(유),2021년 하반기 신입 소프트웨어·하드웨어,"[대학원, 국제의료경영학과, 학점 2.71/4.5]",[Tell us what/who inspires you and why (In Eng...,"[""Innovation and Dynamics of Human Society""\n\..."
4,현대오토에버(주),2022년 상반기 신입 응용프로그래머,"[수도권4년, 공간정보공학과, 학점 3.62/4.5]",[현대오토에버의 해당 직무에 지원한 이유와 앞으로 현대오토에버에서 키워 나갈 커리어...,[공간정보공학과라는 대부분의 사람들에게 생소한 학과에서 공부를 하면서 GIS나 GP...


In [63]:
# csv 저장
result.to_csv('/content/drive/MyDrive/gradu_project/selfs_meta.csv', header=True, index=False, encoding='utf-8')